## BIG G EXPRESS

-Create notebook + csv (csv with notebook name) that can export data to feed into model

-Create py file that does majority of cleaning for you

-Create pipeline that will read and evaluate csv data success

-Create metadata on each run to see what model & data produced best results (money saved)

In [113]:
import pandas as pd
import numpy as np

In [114]:
faults = pd.read_csv("../data/J1939Faults.csv")
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
diagnostics = diagnostics.pivot(index='FaultId', columns='Name', values='Value')
diagnostics.head()

C:\Users\irela\AppData\Local\Temp\ipykernel_6480\2194169026.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")


Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
faults_and_diagnostics = faults.merge(diagnostics, how='left', left_on='RecordID', right_on='FaultId')
faults_and_diagnostics.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
faults_and_diagnostics['FullDerate'] = faults_and_diagnostics['spn'].apply(lambda x: 1 if x == 5246 else 0)
faults_and_diagnostics = faults_and_diagnostics.sort_values(['EquipmentID', 'EventTimeStamp'])

faults_and_diagnostics['EventTimeStamp'] = pd.to_datetime(faults_and_diagnostics['EventTimeStamp'])

faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.where(faults_and_diagnostics['FullDerate']==1)['EventTimeStamp']
faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.groupby('EquipmentID')['NextDerateTime'].fillna(method='bfill')
#faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics['NextDerateTime'].fillna(pd.Timestamp('2019-01-01'))

faults_and_diagnostics['HoursUntilNextDerate'] = (faults_and_diagnostics['NextDerateTime'] - faults_and_diagnostics['EventTimeStamp']).dt.total_seconds()/3600.0
faults_and_diagnostics['DerateInNextTwoHours'] = np.where(faults_and_diagnostics['HoursUntilNextDerate'] <= 2, 1, 0)
faults_and_diagnostics.head()

##ffil nans
# function that does all across the board cleaning and prep

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,FullDerate,NextDerateTime,HoursUntilNextDerate,DerateInNextTwoHours
33355,34467,2092800,2015-04-26 06:16:16,High (Severity High) Engine Speed,NaN,unknown,unknown,unknown,unknown,49,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2019-01-01,32297.728889,0
35080,36192,2123154,2015-04-28 05:29:21,High (Severity High) Engine Speed,NaN,unknown,unknown,unknown,unknown,49,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2019-01-01,32250.510833,0
47186,48298,2346346,2015-05-10 07:11:34,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,11,...,False,NaN,4.728246,3276.75,8.8,0.87,0,2019-01-01,31960.807222,0
47213,48325,2346621,2015-05-10 07:59:25,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,11,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2019-01-01,31960.009722,0
48303,49415,2363162,2015-05-11 13:11:20,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,11,...,False,NaN,4.475814,3276.75,14.4,0.58,0,2019-01-01,31930.811111,0


In [117]:
faults_and_diagnostics.dtypes

RecordID                              int64
ESS_Id                                int64
EventTimeStamp               datetime64[ns]
eventDescription                     object
actionDescription                   float64
ecuSoftwareVersion                   object
ecuSerialNumber                      object
ecuModel                             object
ecuMake                              object
ecuSource                             int64
spn                                   int64
fmi                                   int64
active                                 bool
activeTransitionCount                 int64
faultValue                          float64
EquipmentID                          object
MCTNumber                             int64
Latitude                            float64
Longitude                           float64
LocationTimeStamp                    object
AcceleratorPedal                     object
BarometricPressure                   object
CruiseControlActive             

In [118]:
for col, dtype in {
    "AcceleratorPedal":"float16",
    "BarometricPressure":"float16",
    "CruiseControlActive":"bool",
    "CruiseControlSetSpeed":"float16",
    "DistanceLtd":"float16",
    "EngineCoolantTemperature":"float16",
    "EngineLoad":"float16",
    "EngineOilPressure":"float16",
    "EngineOilTemperature":"float16",
    "EngineRpm":"float16",
    "EngineTimeLtd":"float16",
    "FuelLevel":"float16",
    "FuelLtd":"float32",
    "FuelRate":"float16",
    "FuelTemperature":"float16",
    "IgnStatus":"bool",
    "IntakeManifoldTemperature":"float16",
    "ParkingBrake":"bool",
    "Speed":"float16",
    "SwitchedBatteryVoltage":"float16",
    "Throttle":"float16",
    "TurboBoostPressure":"float16",
    "eventDescription":"str",
    "EquipmentID":"str"
}.items():
    if dtype == 'bool':
        faults_and_diagnostics[col] = faults_and_diagnostics[col].astype('bool')
    else:
        faults_and_diagnostics[col] = pd.to_numeric(faults_and_diagnostics[col], errors='coerce').astype(dtype)

print(faults_and_diagnostics.dtypes)


c:\Users\irela\anaconda3\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\irela\anaconda3\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


RecordID                              int64
ESS_Id                                int64
EventTimeStamp               datetime64[ns]
eventDescription                     object
actionDescription                   float64
ecuSoftwareVersion                   object
ecuSerialNumber                      object
ecuModel                             object
ecuMake                              object
ecuSource                             int64
spn                                   int64
fmi                                   int64
active                                 bool
activeTransitionCount                 int64
faultValue                          float64
EquipmentID                          object
MCTNumber                             int64
Latitude                            float64
Longitude                           float64
LocationTimeStamp                    object
AcceleratorPedal                    float16
BarometricPressure                  float16
CruiseControlActive             

In [119]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lifelines import CoxPHFitter
#from sksurv.ensemble import RandomSurvivalForest

In [120]:
#faults_and_diagnostics = faults_and_diagnostics[(faults_and_diagnostics['spn'] != 5246) & (faults_and_diagnostics['active'] == True)]
faults_and_diagnostics = faults_and_diagnostics[faults_and_diagnostics['active'] == True]

faults_and_diagnostics_train = faults_and_diagnostics[faults_and_diagnostics['EventTimeStamp']<'2019-01-01']
faults_and_diagnostics_test = faults_and_diagnostics[(faults_and_diagnostics['EventTimeStamp']>='2019-01-01') & (faults_and_diagnostics['EventTimeStamp']<='2024-01-01') ]

X_train = faults_and_diagnostics_train.drop(columns=['FuelRate', 'EngineLoad', 'EquipmentID', 'eventDescription', 'LampStatus', 'FullDerate', 'NextDerateTime', 'HoursUntilNextDerate', 'DerateInNextTwoHours', 'RecordID', 'ESS_Id', 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'active', 'faultValue', 'MCTNumber', 'LocationTimeStamp', 'EventTimeStamp', 'activeTransitionCount', 'ServiceDistance'])
X_train = X_train.replace([np.inf, -np.inf], np.nan)

y_train_duration = faults_and_diagnostics_train['HoursUntilNextDerate']
y_train_event = faults_and_diagnostics_train['FullDerate']
X_test = faults_and_diagnostics_test.drop(columns=['FuelRate', 'EngineLoad', 'EquipmentID', 'eventDescription', 'LampStatus', 'FullDerate', 'NextDerateTime', 'HoursUntilNextDerate', 'DerateInNextTwoHours', 'RecordID', 'ESS_Id', 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'active', 'faultValue', 'MCTNumber', 'LocationTimeStamp', 'EventTimeStamp', 'activeTransitionCount', 'ServiceDistance'])
X_test = X_test.replace([np.inf, -np.inf], np.nan)

y_test_duration = faults_and_diagnostics_test['HoursUntilNextDerate']
y_test_event = faults_and_diagnostics_test['FullDerate']

In [121]:
numeric_selector = make_column_selector(dtype_include='number')
categorical_selector = make_column_selector(dtype_include='object')
X_train[categorical_selector] = X_train[categorical_selector].astype(str)
X_test[categorical_selector] = X_test[categorical_selector].astype(str)

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

ct = ColumnTransformer([
    ('num', numeric_transformer, numeric_selector),
    ('cat', categorical_transformer, categorical_selector)
])


X_train_processed = ct.fit_transform(X_train)
if hasattr(X_train_processed, "toarray"):
    X_train_processed = X_train_processed.toarray()
new_feature_names = ct.get_feature_names_out()

X_train_df = pd.DataFrame(X_train_processed, columns=new_feature_names, index=X_train.index)
X_train_df['duration'] = y_train_duration
X_train_df['event'] = y_train_event




In [135]:
pipe = Pipeline([
    ('ct', ct),
    ('model', XGBRegressor(objective='survival:cox', eval_metric='cox-nloglik', tree_method='hist', n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
])

pipe.fit(X_train, y_train_duration)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000022F81602660>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <skle...
                              feature_weights=None, gamma=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [ ]:
y_pred = pipe.predict(X_test)

mae = mean_absolute_error(y_test_duration, y_pred)
mse = mean_squared_error(y_test_duration, y_pred)
r2 = r2_score(y_test_duration, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

Mean Absolute Error (MAE): 4619.035678118757
Mean Squared Error (MSE): 30861526.210379753
R-squared: -2.238874164929541


In [137]:
pipe = Pipeline([
    ('ct', ct),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

pipe.fit(X_train, y_train_duration)
y_pred = pipe.predict(X_test)

mae = mean_absolute_error(y_test_duration, y_pred)
mse = mean_squared_error(y_test_duration, y_pred)
r2 = r2_score(y_test_duration, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

KeyboardInterrupt: 